# CW1 - Amazon Bestsellers Analysis with PySpark


In this assignment you will be tasked with exploring a dataset containing the Top 50 best-selling books from Amazon between 2009-2019. You should complete the exercises presented in the Google Colab Notebook below. This assignment will be graded using CodeGrade.

Exercise 1 (5 Marks): Find the authors with the most entries in the bestseller’s lists, find the number of unique titles for each, the average rating, total number of reviews, and highest position in the ranking.

Exercise 2 (5 Marks): For fiction and non-fiction books, find the average and total number of reviews for the top 10, 25, and 50 of the bestsellers lists, in each year.

Exercise 3 (10 Marks): For each year, find the average price of a fiction and non-fiction book in the top 10, 25 and 50 of the bestsellers lists.

Exercise 4 (10 Marks): For free books—where the price is zero—fine the number of unique titles and authors. Compare the average rating and number of reviews in each year between free and priced books.


In [2]:
# CodeGrade Tag Init1

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# CodeGrade Tag Init2

# Apache Spark uses Java, so first we must install that
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Unpack Spark from google drive
!tar xzf /content/drive/MyDrive/spark-3.3.0-bin-hadoop3.tgz

# Set up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.3.0-bin-hadoop3"

# Install findspark, which helps python locate the psyspark module files
!pip install -q findspark
import findspark
findspark.init()

In [4]:
# Finally, we initialse a "SparkSession", which handles the computations
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

from pyspark.sql import functions as F

In [5]:
# Load the AmazonBooks.csv file into your notebook as a pyspark dataframe

CsvPath = '/content/drive/MyDrive/ADS 2/AmazonBooks-1.csv'

# Load .csv with header, ',' seperators and inferred schema
BooksDF = spark.read\
                     .option('header', 'True')\
                     .option('sep', ',')\
                     .option('inferSchema', 'True')\
                     .csv(CsvPath)

In [6]:
# CodeGrade Tag Init3

BooksDF.printSchema()
BooksDF.show()

root
 |-- Name: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- User Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Rank: integer (nullable = true)

+--------------------+----------------+-----------+-------+-----+----+-----------+----+
|                Name|          Author|User Rating|Reviews|Price|Year|      Genre|Rank|
+--------------------+----------------+-----------+-------+-----+----+-----------+----+
|            The Help|Kathryn Stockett|        4.8|  13871|    6|2009|    Fiction|   1|
|Where the Wild Th...|  Maurice Sendak|        4.8|   9967|   13|2009|    Fiction|   2|
|The Last Olympian...|    Rick Riordan|        4.8|   4628|    7|2009|    Fiction|   3|
|Diary of a Wimpy ...|     Jeff Kinney|        4.8|   3837|   15|2009|    Fiction|   4|
|            Watchmen|      Alan Moore|        4.8|   3829|   42|2009|  

In [7]:
# pyspark.sql.functions contains all the transformations and actions you will
# need
from pyspark.sql import functions as F

# Exercise 1

Find the authors with the most entries in the bestsellers lists. Find the number of unique titles for each author, the average rating, total number of reviews and highest position in the ranking. Create a dataframe where the columns are:

Author, Number of titles, Average Rating, Total Ratings, Highest Position

Sort by the number of titles in descending order.

In [10]:
# CodeGrade Tag Ex1
### Create a dataframe that contains, for each author, the number of unique
### books, the average rating, the number of reviews and the highest rank reached
AuthorDF = BooksDF.groupBy('Author')\
                  .agg(F.countDistinct('Name').alias('Number of titles'),\
                       F.avg('User Rating').alias('Average Rating'),\
                       F.sum('Reviews').alias('Total Ratings'),\
                       F.max('Rank').alias('Highest Position'))

AuthorsDF = AuthorDF.sort(F.desc('Number of titles'))

AuthorsDF.show()

+----------------+----------------+------------------+-------------+----------------+
|          Author|Number of titles|    Average Rating|Total Ratings|Highest Position|
+----------------+----------------+------------------+-------------+----------------+
|     Jeff Kinney|              13| 4.807692307692306|        93529|              20|
|    Rick Riordan|              10|4.7727272727272725|        44169|              24|
| Stephenie Meyer|               8|             4.675|       108273|              26|
|      Dav Pilkey|               7|               4.9|        82541|              10|
|   Bill O'Reilly|               6| 4.642857142857143|        63787|              38|
|    J.K. Rowling|               6|              4.45|        70535|              50|
| Suzanne Collins|               6| 4.666666666666667|       315502|              34|
|       E L James|               5| 4.233333333333333|       178011|              50|
|    John Grisham|               5|               4.4|

# Exercise 2

For fiction and non-fiction books, find the average rating, the average number of reviews, the total number of reviews and the average price in the bestsellers list, for each year. Create a dataframe where the columns are:

Year, Genre, Average Rating, Average Number of Reviews, Total Reviews, Average Price,

Sort by the year in ascending order.

In [12]:
# CodeGrade Tag Ex2
### Create a dataframe that shows the average user rating, average number of
### reviews, total number of reviews and average price in each year of the
### bestsellers list
GenreDF = BooksDF.groupBy('Year', 'Genre')\
                      .agg(F.avg('User Rating').alias('Average Rating'),\
                           F.avg('Reviews').alias('Average Number of Reviews'),\
                           F.sum('Reviews').alias('Total Reviews'),\
                           F.avg('Price').alias('Average Price'))

GenresDF = GenreDF.sort(F.asc('Year'))

GenresDF.show()

+----+-----------+------------------+-------------------------+-------------+------------------+
|Year|      Genre|    Average Rating|Average Number of Reviews|Total Reviews|     Average Price|
+----+-----------+------------------+-------------------------+-------------+------------------+
|2009|    Fiction| 4.591666666666667|        6534.333333333333|       156824|15.583333333333334|
|2009|Non Fiction| 4.576923076923077|        3026.230769230769|        78682| 15.23076923076923|
|2010|    Fiction|             4.615|                  8409.25|       168185|               9.7|
|2010|Non Fiction|4.5200000000000005|       3526.5333333333333|       105796|              16.0|
|2011|    Fiction|4.6190476190476195|       10335.285714285714|       217041|11.619047619047619|
|2011|Non Fiction| 4.513793103448277|        6482.758620689655|       188000|17.620689655172413|
|2012|    Fiction| 4.495238095238096|       19896.238095238095|       417821|12.285714285714286|
|2012|Non Fiction| 4.558620689

# Exercise 3

For each year, find the average price of fiction and non-fiction books in the top 10, 25 and 50 of the bestsellers list. Make a dataframe where the columns are:

Year, Genre, Avg Price in Top 10, Avg Price in Top 25 and Avg Price in Top 50

Sort by the year in ascending order.

In [15]:
# CodeGrade Tag Ex3
### Create a DataFrame that shows the average price for books in the top 10, 25
### and 50 of the bestsellers list, for each year in the dataset
YearDF = BooksDF.groupBy('Year', 'Genre')\
                .agg(F.avg(F.when(F.col('Rank') <= 10,\
                                  F.col('Price'))).alias('Avg Price in Top 10'),\
                     F.avg(F.when(F.col('Rank') <= 25,\
                                  F.col('Price'))).alias('Avg Price in Top 20'),\
                     F.avg(F.when(F.col('Rank') <= 50,\
                                  F.col('Price'))).alias('Avg Price in Top 50'))

YearsDF = YearDF.sort(F.asc('Year'))

YearsDF.show()

+----+-----------+-------------------+-------------------+-------------------+
|Year|      Genre|Avg Price in Top 10|Avg Price in Top 20|Avg Price in Top 50|
+----+-----------+-------------------+-------------------+-------------------+
|2009|    Fiction| 15.833333333333334| 18.866666666666667| 15.583333333333334|
|2009|Non Fiction|               14.5|               19.1|  15.23076923076923|
|2010|    Fiction| 10.777777777777779| 10.928571428571429|                9.7|
|2010|Non Fiction|               16.0| 15.636363636363637|               16.0|
|2011|    Fiction|             10.375| 10.923076923076923| 11.619047619047619|
|2011|Non Fiction|               12.0| 15.666666666666666| 17.620689655172413|
|2012|    Fiction| 13.333333333333334|               11.9| 12.285714285714286|
|2012|Non Fiction|              17.25| 17.933333333333334| 17.482758620689655|
|2013|    Fiction|  9.333333333333334|  9.357142857142858| 10.708333333333334|
|2013|Non Fiction|                8.0| 14.3636363636

# Exercise 4

For free books, find the total number of unique title and author, store these as variables called ```free_titles``` and ```free_authors```.

Compare the average rating and number of reviews for free and priced books, in each year of the dataset. Create a dataframe where the columns are:

Year, Avg Rating Free, Avg Rating Priced, Total Ratings Free, Total Ratings Priced

Sort by the year in ascending order.

In [16]:
# CodeGrade Tag Ex4a
### Find the number of free books in the dataset and the number of authors
### who wrote them
free_titles = BooksDF.filter(F.col('Price') == 0)\
                     .agg(F.countDistinct('Name')\
                     .alias('free_titles'))

free_titles.show()

free_authors = BooksDF.filter(F.col('Price') == 0)\
                      .agg(F.countDistinct('Author')\
                      .alias('free_authors'))

free_authors.show()

+-----------+
|free_titles|
+-----------+
|          9|
+-----------+

+------------+
|free_authors|
+------------+
|           6|
+------------+



In [17]:
# CodeGrade Tag Exb
### Create a dataframe that has, for each year, the average rating and number of
### user reviews for free books and priced books
FreeBookDF = BooksDF.groupBy('Year')\
                     .agg(F.avg(F.when(F.col('Price') == 0, F.col('User Rating'))).alias('Avg Rating Free'),
                          F.avg(F.when(F.col('Price') >= 1, F.col('User Rating'))).alias('Avg Rating Priced'),
                          F.sum(F.when(F.col('Price') == 0, F.col('Reviews'))).alias('Total Reviews Free'),
                          F.sum(F.when(F.col('Price') >= 1, F.col('Reviews'))).alias('Total Reviews Priced'))

FreeBooksDF = FreeBookDF.sort(F.asc('Year'))

FreeBooksDF.show()

+----+---------------+-----------------+------------------+--------------------+
|Year|Avg Rating Free|Avg Rating Priced|Total Reviews Free|Total Reviews Priced|
+----+---------------+-----------------+------------------+--------------------+
|2009|           null|            4.584|              null|              235506|
|2010|            4.6|4.557142857142857|              2122|              271859|
|2011|            4.8|4.553061224489797|              4505|              400536|
|2012|           null|            4.532|              null|              654546|
|2013|            4.8|          4.54375|             33046|              621861|
|2014|           4.75|4.610869565217391|             32738|              760259|
|2015|            4.8|4.644897959183671|             26234|              685435|
|2016|            4.8|4.672916666666665|             29008|              680792|
|2017|            4.8|4.657142857142857|              5836|              638584|
|2018|           null|4.6679